### ⚠️ Note: NVIDIA Driver Installation Required ⚠️

Verify with: ```nvidia-smi```

In [1]:
from matchmaker import matchmaker

In [2]:
engine = matchmaker.MatchingEngine()

In [3]:
engine.load_interactions("data/swipes.csv", 
  decider_col='decidermemberid', 
  other_col='othermemberid', 
  like_col='like', 
  timestamp_col='timestamp')

Data Loaded ✅


In [4]:
engine.run_popularity()

Popularity features added to User DF ✅


In [5]:
engine.user_df.head()

,user_id,pagerank,likes_received,likes_given,like_ratio
0,242983,0.000009,6,251,0.023904
1,185416,0.000004,1,64,0.015625
2,1605941,0.000042,28,84,0.333333
3,1505651,0.000018,11,1029,0.010690
4,43214,0.000006,4,122,0.032787


In [6]:
engine.interactions_df.head()

,decidermemberid,othermemberid,like,timestamp
0,3847776,3227524,1,2021-01-01 00:00:06
1,608590,519321,0,2021-01-01 00:00:06
2,397116,453914,0,2021-01-01 00:00:06
3,3847776,1269455,1,2021-01-01 00:00:06
4,1630969,347909,0,2021-01-01 00:00:23
